In [ ]:
!pip install facenet-pytorch

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader 
import os
import cv2
import matplotlib.pyplot as plt
from IPython.display import FileLink
from sklearn.model_selection import train_test_split
from PIL import Image
from torchvision import models
from torchinfo import summary
from facenet_pytorch import InceptionResnetV1
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import json

* **Cắt ảnh theo bbob để predict**

In [ ]:
df_detect = pd.read_csv("/kaggle/input/private-data-label80/data.csv",index_col = None)
df_detect.head()

In [ ]:
for i in range(len(df_detect)):
    file = df_detect.loc[i,"file_name"]
    img_path = os.path.join("/kaggle/input/80-test/data/",file)
    img = cv2.imread(img_path)
    x,y,w,h = eval(df_detect.loc[i,"bbox"])
    cv2.imwrite(str(i) + ".jpg",img[int(y) : int(y + h), int(x) : int(x + w)])
    

In [ ]:
!zip -r crop_image_80.zip /kaggle/working

In [ ]:
FileLink(r'crop_image_80.zip')

* **Cắt ảnh theo bbob để train**

In [ ]:
df_train = pd.read_csv("/kaggle/input/private-data-label80/data.csv",index_col = None)
df_train.head()

In [ ]:
for i in range(len(df_train)):
    file = df_train.loc[i,"file_name"]
    img_path = os.path.join("/kaggle/input/80-test/data/",file)
    img = cv2.imread(img_path)
    x,y,w,h = eval(df_train.loc[i,"bbox"])
    cv2.imwrite(str(i) + ".jpg",img[int(y) : int(y + h), int(x) : int(x + w)])
    

In [ ]:
!zip -r crop_image.zip /kaggle/working

In [ ]:
FileLink(r'crop_image.zip')

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
df = pd.read_csv("/kaggle/input/d/neetnetnet/label-face/labels.csv", index_col = None)
df.head()

In [ ]:
age_classes = {"Baby" : 0, "Kid" : 1, "Teenager" : 2, "20-30s" : 3, "40-50s" : 4, "Senior" : 5}
gender_classes = {
    label : idx for idx,label in enumerate(df["gender"].unique())
}
emotion_classes = {
    label : idx for idx,label in enumerate(df["emotion"].unique())
}
masked_classes = {
    label : idx for idx,label in enumerate(df["masked"].unique())
}
race_classes = {
    label : idx for idx,label in enumerate(df["race"].unique())
}
skintone_classes = {
    label : idx for idx,label in enumerate(df["skintone"].unique())
}
print(age_classes)
print(gender_classes)
print(emotion_classes)
print(masked_classes)
print(race_classes)
print(skintone_classes)

* **Data train**

In [ ]:
img_paths = []
age_las = []
gender_las = []
emotion_las = []
masked_las = []
race_las = []
skintone_las = []
mask_img_paths = []
root_dir = "/kaggle/input/crop-image"

for i in range(len(df)):
    img_paths.append(os.path.join(root_dir,df.loc[i,"file_name"]))
    mask_img_paths.append(os.path.join(root_dir,df.loc[i,"file_name"]))
    age_las.append(df.loc[i,"age"])
    gender_las.append(df.loc[i,"gender"])
    emotion_las.append(df.loc[i,"emotion"])
    masked_las.append(df.loc[i,"masked"])
    race_las.append(df.loc[i,"race"])
    skintone_las.append(df.loc[i,"skintone"])

age_labels = [age_classes[i] for i in age_las]
gender_labels = [gender_classes[i] for i in gender_las]
emotion_labels = [emotion_classes[i] for i in emotion_las]
masked_labels = [masked_classes[i] for i in masked_las]
race_labels = [race_classes[i] for i in race_las]
skintone_labels = [skintone_classes[i] for i in skintone_las]

* **Data Test 20%**

In [ ]:
test_img_paths = [os.path.join("/kaggle/input/test-img-crop/kaggle/working",i) for i in os.listdir("/kaggle/input/test-img-crop/kaggle/working") ]
img_names = [i for i in os.listdir("/kaggle/input/test-img-crop/kaggle/working")]

* **Data Test 80%**

In [ ]:
test_img_paths = [os.path.join("/kaggle/input/crop-img-test-80/kaggle/working",i) for i in os.listdir("/kaggle/input/crop-img-test-80/kaggle/working") ]
img_names = [i for i in os.listdir("/kaggle/input/crop-img-test-80/kaggle/working")]

In [ ]:
class TrainDataset(Dataset):
    def __init__(
        self, X, y, 
        transform
    ):
        self.transform = transform
        self.img_paths = X
        self.labels = y
    
    def __len__(self):
        return len(self.img_paths)
    
    def __getitem__(self,idx):
        img_path = self.img_paths[idx]
        img = Image.open(img_path).convert("RGB")
        label = self.labels[idx]
        
        img = self.transform(img)
        
        return img,label
        

In [ ]:
class TestDataset(Dataset):
    def __init__(
        self, X, y, 
        transform
    ):
        self.transform = transform
        self.img_paths = X
        self.names = y
    
    def __len__(self):
        return len(self.img_paths)
    
    def __getitem__(self,idx):
        img_path = self.img_paths[idx]
        img = Image.open(img_path).convert("RGB")
        name = self.names[idx]
        
        img = self.transform(img)
        
        return img,name
        

# **Age Model**

In [ ]:
seed = 42
val_size = 0.3
is_shuffle = True
X_train_age, X_val_age, y_train_age, y_val_age = train_test_split(
    img_paths, age_labels, 
    test_size = val_size, 
    random_state = seed,
    shuffle = is_shuffle
)


In [ ]:
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(5),
    transforms.Resize((160,160)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    transforms.RandomErasing(p=0.75,
                             scale=(0.01, 0.3),
                             ratio=(1.0, 1.0),
                             value=0,
                             inplace =True)      
])

val_transform = transforms.Compose([
    transforms.Resize((160,160)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
age_train_dataset = TrainDataset(
    X_train_age, y_train_age,
    transform = train_transform
)

age_val_dataset = TrainDataset(
    X_val_age, y_val_age,
    transform = val_transform
)

In [ ]:
train_batch_size = 64
val_batch_size = 64

age_train_loader = DataLoader(
    age_train_dataset,
    batch_size = train_batch_size,
    shuffle = True
) 

age_val_loader = DataLoader(
    age_val_dataset,
    batch_size = val_batch_size,
    shuffle = False
)

In [ ]:
def evaluation(model,val_loader,criterion):
    model.eval()
    val_loss = 0.0
    running_correct = 0 
    total = 0 
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs,labels)
            val_loss += loss.item()
            
            _, predicted = torch.max(outputs.data,1)
            total += labels.size(0)
            running_correct += (predicted == labels).sum().item()
    
    accuracy = 100 * running_correct / total
    val_loss = val_loss / len(val_loader)
    return val_loss, accuracy

In [ ]:
def save(filename, model):
    filename = "/kaggle/working/" + filename
    torch.save(model.state_dict(), filename)
    print("Saved model as", filename)


* **Age model**

In [ ]:
age_model = InceptionResnetV1(pretrained='vggface2',
                              device=device,
                              classify= True,
                              num_classes=len(age_classes))

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(age_model.parameters(),
                             lr= 1e-4)

* **Train age model**

In [ ]:
train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []
max_epoch = 35
best_accuracy = 50

for epoch in range(max_epoch):
    age_model.train()
    running_loss = 0.0
    running_correct = 0   
    total = 0             
    print(f"Epoch {epoch + 1}: ", end='')
    
    for _ in tqdm(range(len(age_train_loader))):
        inputs, labels = next(iter(age_train_loader))
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = age_model(inputs)
        loss = criterion(outputs, labels)
        running_loss += loss.item()
        loss.backward()
        optimizer.step()  
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        running_correct += (predicted == labels).sum().item()
    epoch_accuracy = 100 * running_correct / total
    epoch_loss = running_loss / len(age_train_loader)
    val_loss, val_accuracy = evaluation(age_model, age_val_loader, criterion)
    print(f"Epoch [{epoch + 1}/{max_epoch}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%")
    if val_accuracy > best_accuracy:
        print("saving at epoch", epoch+1)
        best_accuracy = val_accuracy
        save("age_weight.pt", age_model)
    # save for plot
    train_losses.append(epoch_loss)
    train_accuracies.append(epoch_accuracy)
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)

* **Plot age model**

In [ ]:
plt.plot(train_losses, label='train_losses')
plt.plot(val_losses, label='val_losses')
plt.legend()

In [ ]:
plt.plot(train_accuracies, label='train_accuracy')
plt.plot(val_accuracies, label='val_accuracy')
plt.legend()

* **Predict age model**

In [ ]:
age_model.load_state_dict(torch.load("/kaggle/input/age-weight-1/age_weight.pt"))
age_model.to(device)

In [ ]:
reverse_age = ['Baby', 'Kid', 'Teenager', '20-30s', '40-50s', 'Senior']

In [ ]:
test_transform = transforms.Compose([
    transforms.Resize((160,160)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
test_dataset = TestDataset(
    test_img_paths, img_names,
    transform = test_transform
)

In [ ]:
test_batch_size = 64
test_loader = DataLoader(
    test_dataset,
    batch_size = test_batch_size,
    shuffle = False
)

In [ ]:
age_submiss = {}
age_model.eval()
with torch.no_grad():
    for images, names in test_loader:
        images = images.to(device)
        outputs = age_model(images)
        _, predicted = torch.max(outputs.data,1)
        for predict, name in zip(predicted, names):
            age_submiss[name[:-4]] = reverse_age[predict]



# **Mask Model**

* **Add more mask data from** https://www.kaggle.com/datasets/ashishjangra27/face-mask-12k-images-dataset 

In [ ]:

for img in os.listdir("/kaggle/input/face-mask-12k-images-dataset/Face Mask Dataset/Test/WithMask"):
    mask_img_paths.append(os.path.join("/kaggle/input/face-mask-12k-images-dataset/Face Mask Dataset/Test/WithMask",img))
    masked_labels.append(1)

for img in os.listdir("/kaggle/input/face-mask-12k-images-dataset/Face Mask Dataset/Train/WithMask"):
    mask_img_paths.append(os.path.join("/kaggle/input/face-mask-12k-images-dataset/Face Mask Dataset/Train/WithMask",img))
    masked_labels.append(1)
    
for img in os.listdir("/kaggle/input/face-mask-12k-images-dataset/Face Mask Dataset/Validation/WithMask"):
    mask_img_paths.append(os.path.join("/kaggle/input/face-mask-12k-images-dataset/Face Mask Dataset/Validation/WithMask",img))
    masked_labels.append(1)

In [ ]:
len(mask_img_paths)

In [ ]:
seed = 42
val_size = 0.3
is_shuffle = True
X_train_mask, X_val_mask, y_train_mask, y_val_mask = train_test_split(
    mask_img_paths, masked_labels, 
    test_size = val_size, 
    random_state = seed,
    shuffle = is_shuffle
)

In [ ]:
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(5),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    transforms.RandomErasing(p=0.75,
                             scale=(0.01, 0.3),
                             ratio=(1.0, 1.0),
                             value=0,
                             inplace =True)      
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
mask_train_dataset = TrainDataset(
    X_train_mask, y_train_mask,
    transform = train_transform
)

mask_val_dataset = TrainDataset(
    X_val_mask, y_val_mask,
    transform = val_transform
)

In [ ]:
train_batch_size = 64
val_batch_size = 64

mask_train_loader = DataLoader(
    mask_train_dataset,
    batch_size = train_batch_size,
    shuffle = True
) 

mask_val_loader = DataLoader(
    mask_val_dataset,
    batch_size = val_batch_size,
    shuffle = False
)

In [ ]:
def evaluation(model,val_loader,criterion):
    model.eval()
    val_loss = 0.0
    running_correct = 0 
    total = 0 
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs,labels)
            val_loss += loss.item()
            
            _, predicted = torch.max(outputs.data,1)
            total += labels.size(0)
            running_correct += (predicted == labels).sum().item()
    
    accuracy = 100 * running_correct / total
    val_loss = val_loss / len(val_loader)
    return val_loss, accuracy

In [ ]:
def save(filename, model):
    filename = "/kaggle/working/" + filename
    torch.save(model.state_dict(), filename)
    print("Saved model as", filename)


* **Mask model**

In [ ]:
mask_model = InceptionResnetV1(pretrained='vggface2',
                              device=device,
                              classify= True,
                              num_classes=2)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(mask_model.parameters(),
                             lr= 1e-4)

* **Train mask model**

In [ ]:
train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []
max_epoch = 50
best_accuracy = 50
for epoch in range(max_epoch):
    mask_model.train()
    running_loss = 0.0
    running_correct = 0   
    total = 0             
    print(f"Epoch {epoch + 1}: ", end='')
    
    for _ in tqdm(range(len(mask_train_loader))):
        inputs, labels = next(iter(mask_train_loader))
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = mask_model(inputs)
        loss = criterion(outputs, labels)
        running_loss += loss.item()
        loss.backward()
        optimizer.step()  
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        running_correct += (predicted == labels).sum().item()
    epoch_accuracy = 100 * running_correct / total
    epoch_loss = running_loss / len(mask_train_loader)
    val_loss, val_accuracy = evaluation(mask_model, mask_val_loader, criterion)
    print(f"Epoch [{epoch + 1}/{max_epoch}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%")
    if val_accuracy > best_accuracy:
        print("saving at epoch", epoch+1)
        best_accuracy = val_accuracy
        save("mask_weight.pt", mask_model)
    # save for plot
    train_losses.append(epoch_loss)
    train_accuracies.append(epoch_accuracy)
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)

* **Plot mask model**

In [ ]:
plt.plot(train_losses, label='train_losses')
plt.plot(val_losses, label='val_losses')
plt.legend()

In [ ]:
plt.plot(train_accuracies, label='train_accuracy')
plt.plot(val_accuracies, label='val_accuracy')
plt.legend()

* **Predict mask model**

In [ ]:
mask_model.load_state_dict(torch.load("/kaggle/input/weight-mask/mask_weight.pt"))
mask_model.to(device)

In [ ]:
reverse_mask = ['unmasked', 'masked']

In [ ]:
test_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
test_dataset = TestDataset(
    test_img_paths, img_names,
    transform = test_transform
)

In [ ]:
test_batch_size = 64
test_loader = DataLoader(
    test_dataset,
    batch_size = test_batch_size,
    shuffle = False
)

In [ ]:
mask_submiss = {}
mask_model.eval()
with torch.no_grad():
    for images, names in test_loader:
        images = images.to(device)
        outputs = mask_model(images)
        _, predicted = torch.max(outputs.data,1)
        for predict, name in zip(predicted, names):
            mask_submiss[name[:-4]] = reverse_mask[predict]



# **Skintone model**

In [ ]:
seed = 42
val_size = 0.3
is_shuffle = True
X_train_skintone, X_val_skintone, y_train_skintone, y_val_skintone = train_test_split(
    img_paths, skintone_labels, 
    test_size = val_size, 
    random_state = seed,
    shuffle = is_shuffle
)



In [ ]:
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(5),
    transforms.Resize((160,160)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    transforms.RandomErasing(p=0.75,
                             scale=(0.01, 0.3),
                             ratio=(1.0, 1.0),
                             value=0,
                             inplace =True)      
])

val_transform = transforms.Compose([
    transforms.Resize((160,160)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
skintone_train_dataset = TrainDataset(
    X_train_skintone, y_train_skintone,
    transform = train_transform
)

skintone_val_dataset = TrainDataset(
    X_val_skintone, y_val_skintone,
    transform = val_transform
)

In [ ]:
train_batch_size = 64
val_batch_size = 64

skintone_train_loader = DataLoader(
    skintone_train_dataset,
    batch_size = train_batch_size,
    shuffle = True
) 

skintone_val_loader = DataLoader(
    skintone_val_dataset,
    batch_size = val_batch_size,
    shuffle = False
)

In [ ]:
def evaluation(model,val_loader,criterion):
    model.eval()
    val_loss = 0.0
    running_correct = 0 
    total = 0 
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs,labels)
            val_loss += loss.item()
            
            _, predicted = torch.max(outputs.data,1)
            total += labels.size(0)
            running_correct += (predicted == labels).sum().item()
    
    accuracy = 100 * running_correct / total
    val_loss = val_loss / len(val_loader)
    return val_loss, accuracy

In [ ]:
def save(filename, model):
    filename = "/kaggle/working/" + filename
    torch.save(model.state_dict(), filename)
    print("Saved model as", filename)


* **Skintone model**

In [ ]:
skintone_model = InceptionResnetV1(pretrained='vggface2',
                              device=device,
                              classify= True,
                              num_classes=len(skintone_classes))

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(skintone_model.parameters(),
                             lr= 1e-4)

* **Train skintone model**

In [ ]:
train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []
max_epoch = 25
best_accuracy = 50

for epoch in range(max_epoch):
    skintone_model.train()
    running_loss = 0.0
    running_correct = 0   
    total = 0             
    print(f"Epoch {epoch + 1}: ", end='')
    
    for _ in tqdm(range(len(skintone_train_loader))):
        inputs, labels = next(iter(skintone_train_loader))
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = skintone_model(inputs)
        loss = criterion(outputs, labels)
        running_loss += loss.item()
        loss.backward()
        optimizer.step()  
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        running_correct += (predicted == labels).sum().item()
    epoch_accuracy = 100 * running_correct / total
    epoch_loss = running_loss / len(skintone_train_loader)
    val_loss, val_accuracy = evaluation(skintone_model, skintone_val_loader, criterion)
    print(f"Epoch [{epoch + 1}/{max_epoch}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%")
    if val_accuracy > best_accuracy:
        print("saving at epoch", epoch+1)
        best_accuracy = val_accuracy
        save("skintone_weight.pt", skintone_model)
    # save for plot
    train_losses.append(epoch_loss)
    train_accuracies.append(epoch_accuracy)
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)

* **Plot skintone model**

In [ ]:
plt.plot(train_losses, label='train_losses')
plt.plot(val_losses, label='val_losses')
plt.legend()

In [ ]:
plt.plot(train_accuracies, label='train_accuracy')
plt.plot(val_accuracies, label='val_accuracy')
plt.legend()

* **Predict skintone model**

In [ ]:
skintone_model.load_state_dict(torch.load("/kaggle/input/skintone-weight-1/skintone_weight.pt"))
skintone_model.to(device)

In [ ]:
reverse_skintone = ['mid-light', 'light', 'mid-dark', 'dark']

In [ ]:
test_transform = transforms.Compose([
    transforms.Resize((160,160)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
test_dataset = TestDataset(
    test_img_paths, img_names,
    transform = test_transform
)

In [ ]:
test_batch_size = 64
test_loader = DataLoader(
    test_dataset,
    batch_size = test_batch_size,
    shuffle = False
)

In [ ]:
skintone_submiss = {}
skintone_model.eval()
with torch.no_grad():
    for images, names in test_loader:
        images = images.to(device)
        outputs = skintone_model(images)
        _, predicted = torch.max(outputs.data,1)
        for predict, name in zip(predicted, names):
            skintone_submiss[name[:-4]] = reverse_skintone[predict]



# **Race model**

In [ ]:
seed = 42
val_size = 0.3
is_shuffle = True
X_train_race, X_val_race, y_train_race, y_val_race = train_test_split(
    img_paths, race_labels, 
    test_size = val_size, 
    random_state = seed,
    shuffle = is_shuffle
)

In [ ]:
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(5),
    transforms.Resize((160,160)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    transforms.RandomErasing(p=0.75,
                             scale=(0.01, 0.3),
                             ratio=(1.0, 1.0),
                             value=0,
                             inplace =True)      
])

val_transform = transforms.Compose([
    transforms.Resize((160,160)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
race_train_dataset = TrainDataset(
    X_train_race, y_train_race,
    transform = train_transform
)

race_val_dataset = TrainDataset(
    X_val_race, y_val_race,
    transform = val_transform
)

In [ ]:
train_batch_size = 32
val_batch_size = 32

race_train_loader = DataLoader(
    race_train_dataset,
    batch_size = train_batch_size,
    shuffle = True
) 

race_val_loader = DataLoader(
    race_val_dataset,
    batch_size = val_batch_size,
    shuffle = False
)

In [ ]:
def evaluation(model,val_loader,criterion):
    model.eval()
    mpred, mlabel = [], []
    val_loss = 0.0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs,labels)
            val_loss += loss.item()
            
            _, predicted = torch.max(outputs.data,1)
            mpred.extend(predicted.tolist())
            mlabel.extend(labels.tolist())
    
    accuracy = 100 * accuracy_score(mpred, mlabel)
    val_loss = val_loss / len(val_loader)
    return val_loss, accuracy

In [ ]:
def save(filename, model):
    filename = "/kaggle/working/" + filename
    torch.save(model.state_dict(), filename)
    print("Saved model as", filename)

* **Race model**

In [ ]:
race_model = InceptionResnetV1(pretrained='vggface2',
                              device=device,
                              classify= True,
                              num_classes=len(race_classes))

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(race_model.parameters(),
                             lr=1e-4)

* **Train race model**

In [ ]:
train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []
max_epoch = 20
best_accuracy = 10
for epoch in range(max_epoch):
    race_model.train()
    mpred, mlabel = [], []
    running_loss = 0.0      
    
    print(f"Epoch {epoch + 1}: ", end='')
    for _ in tqdm(range(len(race_train_loader))):
        inputs, labels = next(iter(race_train_loader))
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = race_model(inputs)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        running_loss += loss.item()
        
        loss.backward()
        optimizer.step()  
    
        
        _, predicted = torch.max(outputs.data, 1)
        mpred.extend(predicted.tolist())
        mlabel.extend(labels.tolist())

    epoch_accuracy = 100 * accuracy_score(mpred, mlabel)
    epoch_loss = running_loss / (len(race_train_loader))
    
    val_loss, val_accuracy = evaluation(race_model, race_val_loader, criterion)
    print(f"Epoch [{epoch + 1}/{max_epoch}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%")
        
    if val_accuracy > best_accuracy:
        print("saving at epoch", epoch+1)
        best_accuracy = val_accuracy
        save("race_weight.pt", race_model)
    
    # save for plot
    train_losses.append(epoch_loss)
    train_accuracies.append(epoch_accuracy)
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)

* **Predict race model**

In [ ]:
race_model.load_state_dict(torch.load("/kaggle/input/face-analysis-weights/race_weightIRNV1.pt"))
race_model.to(device)

In [ ]:
reverse_race = ['Caucasian', 'Mongoloid', 'Negroid']

In [ ]:
test_transform = transforms.Compose([
    transforms.Resize((160,160)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
test_dataset = TestDataset(
    test_img_paths, img_names,
    transform = test_transform
)

In [ ]:
test_batch_size = 64
test_loader = DataLoader(
    test_dataset,
    batch_size = test_batch_size,
    shuffle = False
)

In [ ]:
race_submiss = {}
race_model.eval()
with torch.no_grad():
    for images, names in test_loader:
        images = images.to(device)
        outputs = race_model(images)
        _, predicted = torch.max(outputs.data,1)
        for predict, name in zip(predicted, names):
            race_submiss[name[:-4]] = reverse_race[predict]



# **Gender model**

In [ ]:
seed = 42
val_size = 0.3
is_shuffle = True
X_train_gender, X_val_gender, y_train_gender, y_val_gender = train_test_split(
    img_paths, gender_labels, 
    test_size = val_size, 
    random_state = seed,
    shuffle = is_shuffle
)

In [ ]:
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(5),
    transforms.Resize((160,160)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    transforms.RandomErasing(p=0.75,
                             scale=(0.01, 0.3),
                             ratio=(1.0, 1.0),
                             value=0,
                             inplace =True)      
])

val_transform = transforms.Compose([
    transforms.Resize((160,160)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
gender_train_dataset = TrainDataset(
    X_train_gender, y_train_gender,
    transform = train_transform
)

gender_val_dataset = TrainDataset(
    X_val_gender, y_val_gender,
    transform = val_transform
)

In [ ]:
train_batch_size = 32
val_batch_size = 32

gender_train_loader = DataLoader(
    gender_train_dataset,
    batch_size = train_batch_size,
    shuffle = True
) 

gender_val_loader = DataLoader(
    gender_val_dataset,
    batch_size = val_batch_size,
    shuffle = False
)

In [ ]:
def evaluation(model,val_loader,criterion):
    model.eval()
    mpred, mlabel = [], []
    val_loss = 0.0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs[:, 1],labels.float())
            val_loss += loss.item()
            
            _, predicted = torch.max(outputs.data,1)
            mpred.extend(predicted.tolist())
            mlabel.extend(labels.tolist())
    
    accuracy = 100 * accuracy_score(mpred, mlabel)
    val_loss = val_loss / len(val_loader)
    return val_loss, accuracy

In [ ]:
def save(filename, model):
    filename = "/kaggle/working/" + filename
    torch.save(model.state_dict(), filename)
    print("Saved model as", filename)


* **Gender model**

In [ ]:
gender_model = InceptionResnetV1(pretrained='vggface2',
                              device=device,
                              classify= True,
                              num_classes=len(gender_classes))

In [ ]:
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(gender_model.parameters(),
                             lr=1e-4,
                             weight_decay=5e-7)

* **Train gender model**

In [ ]:
train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []
max_epoch = 20
best_accuracy = 10
for epoch in range(max_epoch):
    gender_model.train()
    mpred, mlabel = [], []
    running_loss = 0.0      
    
    print(f"Epoch {epoch + 1}: ", end='')
    for _ in tqdm(range(len(gender_train_loader))):
        inputs, labels = next(iter(gender_train_loader))
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = gender_model(inputs)
        loss = criterion(outputs[:, 1], labels.float())
        optimizer.zero_grad()
        running_loss += loss.item()
        
        loss.backward()
        optimizer.step()  
    
        
        _, predicted = torch.max(outputs.data, 1)
        mpred.extend(predicted.tolist())
        mlabel.extend(labels.tolist())

    epoch_accuracy = 100 * accuracy_score(mpred, mlabel)
    epoch_loss = running_loss / (len(gender_train_loader))
    
    val_loss, val_accuracy = evaluation(gender_model, gender_val_loader, criterion)
    print(f"Epoch [{epoch + 1}/{max_epoch}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%")
        
    if val_accuracy > best_accuracy:
        print("saving at epoch", epoch+1)
        best_accuracy = val_accuracy
        save("gender_weight.pt", gender_model)
    
    # save for plot
    train_losses.append(epoch_loss)
    train_accuracies.append(epoch_accuracy)
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)

* **Plot gender model**

In [ ]:
plt.plot(train_losses, label='train_losses')
plt.plot(val_losses, label='val_losses')
plt.legend()

In [ ]:
plt.plot(train_accuracies, label='train_accuracy')
plt.plot(val_accuracies, label='val_accuracy')
plt.legend()

* **Predict gender model**

In [ ]:
gender_model.load_state_dict(torch.load("/kaggle/input/face-analysis-weights/gender_weightIRNV1.pt"))
gender_model.to(device)

In [ ]:
reverse_gender = ['Male', 'Female']

In [ ]:
test_transform = transforms.Compose([
    transforms.Resize((160,160)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
test_dataset = TestDataset(
    test_img_paths, img_names,
    transform = test_transform
)

In [ ]:
test_batch_size = 64
test_loader = DataLoader(
    test_dataset,
    batch_size = test_batch_size,
    shuffle = False
)

In [ ]:
gender_submiss = {}
gender_model.eval()
with torch.no_grad():
    for images, names in test_loader:
        images = images.to(device)
        outputs = gender_model(images)
        _, predicted = torch.max(outputs.data,1)
        for predict, name in zip(predicted, names):
            gender_submiss[name[:-4]] = reverse_gender[predict]



# **Emotion model**

In [ ]:
seed = 42
val_size = 0.3
is_shuffle = True
X_train_emotion, X_val_emotion, y_train_emotion, y_val_emotion = train_test_split(
    img_paths,
    emotion_labels, 
    test_size = val_size, 
    random_state = seed,
    shuffle = is_shuffle
)

In [ ]:
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(20),
#     transforms.ColorJitter(brightness=(0.1,0.6), contrast=1,saturation=0, hue=0.4),
    transforms.Resize((160,160)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    transforms.RandomAffine(
        degrees=0,
        translate=(0.01, 0.12),
        shear=(0.01, 0.03),
    ),
    transforms.RandomErasing(p=0.7,
                             scale=(0.01, 0.3),
                             ratio=(1.0, 1.0),
                             value=0,
                             inplace =True)      
])

val_transform = transforms.Compose([
    transforms.Resize((160,160)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
emotion_train_dataset = TrainDataset(
    X_train_emotion, y_train_emotion,
    transform = train_transform
)

emotion_val_dataset = TrainDataset(
    X_val_emotion, y_val_emotion,
    transform = val_transform
)

In [ ]:
train_batch_size = 64
val_batch_size = 64

emotion_train_loader = DataLoader(
    emotion_train_dataset,
    batch_size = train_batch_size,
    shuffle = True
) 

emotion_val_loader = DataLoader(
    emotion_val_dataset,
    batch_size = val_batch_size,
    shuffle = False
)

In [ ]:
def evaluation(model,val_loader,mcriterion):
    model.eval()
    mpreds, mlabels = [], []
    mval_loss = 0.0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            mloss = mcriterion(outputs,labels)
            mval_loss += mloss.item()
            
            _, predicted = torch.max(outputs.data,1)
            mpreds.extend(predicted.tolist())
            mlabels.extend(labels.tolist())
    
    maccuracy = 100 * accuracy_score(mpreds, mlabels)
    mval_loss = mval_loss / len(val_loader)
    return mval_loss, maccuracy

In [ ]:
def save(filename, model):
    filename = "/kaggle/working/" + filename
    torch.save(model.state_dict(), filename)
    print("Saved model as", filename)

* **Emotion model**

In [ ]:
emotion_model = InceptionResnetV1(pretrained='vggface2',
                              device=device,
                              classify= True,
                              num_classes=len(emotion_classes))

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(emotion_model.parameters(),
                             lr=1e-4)

* **Train emotion model**

In [ ]:
train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []
max_epoch = 10
best_accuracy = 10
for epoch in range(max_epoch):
    emotion_model.train()
    mpred, mlabel = [], []
    running_loss = 0.0      
    
    print(f"Epoch {epoch + 1}: ", end='')
    for _ in tqdm(range(len(emotion_train_loader))):
        inputs, labels = next(iter(emotion_train_loader))
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = emotion_model(inputs)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        running_loss += loss.item()
        
        loss.backward()
        optimizer.step()  
    
        
        _, predicted = torch.max(outputs.data, 1)
        mpred.extend(predicted.tolist())
        mlabel.extend(labels.tolist())

    epoch_accuracy = 100 * accuracy_score(mpred, mlabel)
    epoch_loss = running_loss / (len(emotion_train_loader))
    
    val_loss, val_accuracy = evaluation(emotion_model, emotion_val_loader, criterion)
    print(f"Epoch [{epoch + 1}/{max_epoch}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%")
        
    if val_accuracy > best_accuracy:
        print("saving at epoch", epoch+1)
        best_accuracy = val_accuracy
        save("emotion_weight.pt", emotion_model)
    
    # save for plot
    train_losses.append(epoch_loss)
    train_accuracies.append(epoch_accuracy)
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)

* **Predict emotion model**

In [ ]:
emotion_model.load_state_dict(torch.load("/kaggle/input/face-analysis-weights/emotion_weightIRNV1.pt"))
emotion_model.to(device)

In [ ]:
reverse_emotion = ['Neutral', 'Happiness', 'Anger', 'Surprise', 'Fear', 'Sadness', 'Disgust']

In [ ]:
test_transform = transforms.Compose([
    transforms.Resize((160,160)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
test_dataset = TestDataset(
    test_img_paths, img_names,
    transform = test_transform
)

In [ ]:
test_batch_size = 64
test_loader = DataLoader(
    test_dataset,
    batch_size = test_batch_size,
    shuffle = False
)

In [ ]:
emotion_submiss = {}
emotion_model.eval()
with torch.no_grad():
    for images, names in test_loader:
        images = images.to(device)
        outputs = emotion_model(images)
        _, predicted = torch.max(outputs.data,1)
        for predict, name in zip(predicted, names):
            emotion_submiss[name[:-4]] = reverse_emotion[predict]


# **File Submission**

In [ ]:
df1 = pd.read_csv("/kaggle/input/example1/answer.csv")
df1.head()

# for i in range(len(df1)):
#     x, y, w, h = eval(df1.loc[i,'bbox'])
#     df1.loc[i,'bbox'] = str([int(x), int(y), int(w), int(h)])

In [ ]:
df1.insert(3, "image_id", 1)

In [ ]:
with open("/kaggle/input/name-to-img-id/file_name_to_image_id.json") as f:
    anhxa = json.load(f)

In [ ]:
for i in anhxa:
    df1.loc[df1["file_name"] == i, "image_id"]  = anhxa[i]

In [ ]:
df1 = df1.drop(columns="Unnamed: 0")

In [ ]:
for i in race_submiss:
    df1.loc[int(i),"race"] = race_submiss[i]
for i in age_submiss:
    df1.loc[int(i),"age"] = age_submiss[i]
for i in emotion_submiss:
    df1.loc[int(i),"emotion"] = emotion_submiss[i]
for i in gender_submiss:
    df1.loc[int(i),"gender"] = gender_submiss[i]
for i in skintone_submiss:
    df1.loc[int(i),"skintone"] = skintone_submiss[i]
for i in mask_submiss:
    df1.loc[int(i),"masked"] = mask_submiss[i]


In [ ]:
df1.head()

In [ ]:
df1.to_csv("answer.csv", index=False )

In [ ]:
FileLink(r'answer.csv')